In [65]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

# Set your Spotify API credentials
client_id = ‘CLIENT_ID’
client_secret = ‘CLIENT_SECRET’
redirect_uri = ‘REDIRECT_URI’
username = ‘USERNAME’

client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [71]:
test = sp.search(q='artist:' + "Parquet Courts", type='artist')
print(test)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AParquet+Courts&type=artist&offset=0&limit=10', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/23NIwARd4vPbxt3wwNnJ6k'}, 'followers': {'href': None, 'total': 275852}, 'genres': ['alternative rock', 'american post-punk', 'brooklyn indie', 'chamber pop', 'crank wave', 'denton tx indie', 'indie rock', 'lo-fi', 'neo-psychedelic', 'noise pop', 'noise rock'], 'href': 'https://api.spotify.com/v1/artists/23NIwARd4vPbxt3wwNnJ6k', 'id': '23NIwARd4vPbxt3wwNnJ6k', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eba968b1bb3c46e65911d8bb8c', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174a968b1bb3c46e65911d8bb8c', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178a968b1bb3c46e65911d8bb8c', 'width': 160}], 'name': 'Parquet Courts', 'popularity': 47, 'type': 'artist', 'uri': 'spotify:artist:23NIwARd4vPbxt3wwNnJ6k'}], 'limi

In [66]:
api_cache = {}

In [80]:
import time
from spotipy.client import SpotifyException

df = pd.read_csv('./reviews_data.csv')

# Function to extract the artist name before the comma
def extract_artist_name(full_name):
    return full_name.split(',')[0].strip()

def spotify_query(artist_name):
    try:
        # Get spotify data for this artist
        artist_data = {}
        results_spotify = sp.search(q='artist:' + artist_name, type='artist')
        artist = results_spotify['artists']['items'][0]
        genres = artist['genres']            
        popularity = artist['popularity']
        uri = artist['uri']

        # Update cache
        api_cache[artist_name] = {'genres': genres, 'popularity': popularity, 'uri': uri}
        artist_data = {'genres': genres, 'popularity': popularity, 'uri': uri}

    except (IndexError, SpotifyException) as e:
        if ',' in artist_name:
            # Case where multiple artists were listed, used only first artist's stats
            new_artist_name = extract_artist_name(artist_name)
            artist_data = spotify_query(new_artist_name)
        else:
            artist_data = {'genres': None, 'popularity': None, 'uri': None}
                
    return artist_data

# Function to get genre, popularity, and URI for an artist
def get_artist_info(artist_names):
    results = []
    for artist_name in artist_names:
        if artist_name in api_cache:
            results.append(api_cache[artist_name])
        elif type(artist_name) != str:
            print(f"ARTIST NAME: {artist_name} is nan?")
            continue
        else:
            artist_data = spotify_query(artist_name)
            results.append(artist_data)

    return results

# Specify batch size for queries
batch_size = 50  # Adjust as needed
starting_index = 0 # If running multiple times is needed to avoid API rate limit


df['genre'], df['popularity'], df['uri'] = zip(*df['name'].apply(lambda x: (None, None, None)))

# Iterate over the DataFrame in batches
for i in range(starting_index, len(df), batch_size):
    print(f"BATCH NUMBER {i}")
    batch_df = df.iloc[i:i + batch_size]

    # Get artist information for the current batch
    artist_names = batch_df['name'].tolist()
    results = get_artist_info(artist_names)

    # Update spreadsheet
    for j, result in enumerate(results):
        df.at[i + j, 'genre'] = result['genres']
        df.at[i + j, 'popularity'] = result['popularity']
        df.at[i + j, 'uri'] = result['uri']
    output_filename = 'albums_with_info.csv'
    batch_df.to_csv(output_filename, mode='a', index=False, header=not i, encoding='utf-8')

    time.sleep(1)

BATCH NUMBER 3050
BATCH NUMBER 3100
BATCH NUMBER 3150
BATCH NUMBER 3200
BATCH NUMBER 3250
BATCH NUMBER 3300
BATCH NUMBER 3350
BATCH NUMBER 3400
BATCH NUMBER 3450
BATCH NUMBER 3500
BATCH NUMBER 3550
BATCH NUMBER 3600
BATCH NUMBER 3650
BATCH NUMBER 3700
BATCH NUMBER 3750
BATCH NUMBER 3800
BATCH NUMBER 3850
BATCH NUMBER 3900
BATCH NUMBER 3950
BATCH NUMBER 4000
BATCH NUMBER 4050
BATCH NUMBER 4100
BATCH NUMBER 4150
BATCH NUMBER 4200
BATCH NUMBER 4250
BATCH NUMBER 4300
BATCH NUMBER 4350
BATCH NUMBER 4400
BATCH NUMBER 4450
BATCH NUMBER 4500


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Planningtorock, Planningtorock, Planningtorock, Planningtorock, Planningtorock, Planningtorock, Planningtorock, Planningtorock, Planningtorock, The Knife, The Knife, The Knife, The Knife, The Knife, The Knife, The Knife, The Knife, The Knife, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims, Mt. Sims', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to Bad request.


BATCH NUMBER 4550
BATCH NUMBER 4600
BATCH NUMBER 4650
BATCH NUMBER 4700
BATCH NUMBER 4750
BATCH NUMBER 4800
BATCH NUMBER 4850
BATCH NUMBER 4900
BATCH NUMBER 4950
BATCH NUMBER 5000
BATCH NUMBER 5050
BATCH NUMBER 5100
BATCH NUMBER 5150
BATCH NUMBER 5200
BATCH NUMBER 5250
BATCH NUMBER 5300
BATCH NUMBER 5350
BATCH NUMBER 5400
BATCH NUMBER 5450
BATCH NUMBER 5500
BATCH NUMBER 5550
BATCH NUMBER 5600


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle, Throbbing Gristle', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to Bad request.


BATCH NUMBER 5650
BATCH NUMBER 5700
BATCH NUMBER 5750
BATCH NUMBER 5800
BATCH NUMBER 5850
BATCH NUMBER 5900
BATCH NUMBER 5950
BATCH NUMBER 6000
BATCH NUMBER 6050
BATCH NUMBER 6100
BATCH NUMBER 6150
BATCH NUMBER 6200


...

BATCH NUMBER 18100
BATCH NUMBER 18150
BATCH NUMBER 18200
BATCH NUMBER 18250
BATCH NUMBER 18300
BATCH NUMBER 18350
BATCH NUMBER 18400
BATCH NUMBER 18450
BATCH NUMBER 18500
BATCH NUMBER 18550
BATCH NUMBER 18600
BATCH NUMBER 18650
BATCH NUMBER 18700
BATCH NUMBER 18750
BATCH NUMBER 18800
BATCH NUMBER 18850
BATCH NUMBER 18900
BATCH NUMBER 18950
BATCH NUMBER 19000
BATCH NUMBER 19050
BATCH NUMBER 19100
BATCH NUMBER 19150
BATCH NUMBER 19200
BATCH NUMBER 19250
BATCH NUMBER 19300
BATCH NUMBER 19350
BATCH NUMBER 19400
BATCH NUMBER 19450
BATCH NUMBER 19500
BATCH NUMBER 19550
BATCH NUMBER 19600
BATCH NUMBER 19650
BATCH NUMBER 19700
BATCH NUMBER 19750
BATCH NUMBER 19800
BATCH NUMBER 19850
BATCH NUMBER 19900
BATCH NUMBER 19950
BATCH NUMBER 20000
BATCH NUMBER 20050
BATCH NUMBER 20100
BATCH NUMBER 20150
BATCH NUMBER 20200
BATCH NUMBER 20250
BATCH NUMBER 20300
BATCH NUMBER 20350
BATCH NUMBER 20400
BATCH NUMBER 20450
BATCH NUMBER 20500
BATCH NUMBER 20550
BATCH NUMBER 20600
BATCH NUMBER 20650
BATCH NUMBER

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists, Various Artists', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to Bad request.


BATCH NUMBER 21100
BATCH NUMBER 21150


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:“Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic, “Weird Al” Yankovic', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to Bad request.


BATCH NUMBER 21200
BATCH NUMBER 21250
BATCH NUMBER 21300
BATCH NUMBER 21350
